In [ ]:
import requests
from bs4 import BeautifulSoup

from typing import Literal
from urllib.parse import urlencode
import csv
from datetime import datetime
import time

In [ ]:
class EbayScraping:

    @staticmethod
    def build_search_url(
        query,
        page=1,
        category=0,
        items_per_page: Literal[60,120,240] = 120,
        sort_order:  Literal["best_match", "ending_soonest", "newly_listed"] = "newly_listed"
        ):

        SORT_ORDER_MAP = {
            "best_match": 12,
            "ending_soonest": 1,
            "newly_listed": 10,
        }

        return "https://www.ebay.com/sch/i.html?" + urlencode(
            {
                "_nkw": query,
                "_sacat": category,
                "_ipg": items_per_page,
                "_sop": SORT_ORDER_MAP[sort_order],
                "_pgn": page
            }
        )

    @staticmethod
    def fetch_page_content(url, headers=None):
        if headers is None:
            headers = {
                'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0',
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
                "Accept-Language": "en-US,en;q=0.9",
                "Accept-Encoding": "gzip, deflate, br",
                    }
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()

        except requests.RequestException as e:
            print(f'Error fetching the page: {e}')
            return None

        soup = BeautifulSoup(response.content,'html.parser')

        return soup

    @staticmethod
    def parse_listing_data(soup, parsed_listings: list= [], container_tag='div',container_class='s-item__info clearfix'):

        item_containers = soup.find_all(container_tag, class_=container_class)

        for container in item_containers:
            try:

                title = container.find('div', class_='s-item__title').text.strip()

                if title == 'Shop on eBay':
                    """remove hidden items to prevent scraping"""
                    continue

                parsed_listings.append(
                    {
                        'title': container.find('div', class_='s-item__title').text.strip(),
                        'price': container.find('span', class_='s-item__price').text.strip(),
                        'shipping': container.find('span', class_='s-item__shipping').text.strip(),
                        'subtitles': container.find('div', class_='s-item__subtitle').text.strip(),
                        'listing_date': container.find('span', class_='s-item__dynamic s-item__listingDate').text.strip(),
                        'seller_name': container.find('span', class_='s-item__seller-info-text').text.strip().split(' ', 1)[0],
                        }
                    )

            except Exception as e:
                pass

        return parsed_listings

    @staticmethod
    def save_to_csv(data, filename):

        fieldnames = data[0].keys()

        try:
            with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                writer.writeheader()
                for row in data:
                    writer.writerow(row)


            print(f"Data successfully saved to {filename}")

        except IOError:
            print(f"I/O error occurred while writing to {filename}")

        except Exception as e:
            print(f"An error occurred: {str(e)}")

    @classmethod
    def scrape_search_results(
        cls,
        query,
        max_pages=1,
        category=0,
        items_per_page=120,
        sort_order:  Literal["best_match", "ending_soonest", "newly_listed"] = "newly_listed") -> None:

        listing_data = []

        page_url = cls.build_search_url(query, page=1, category=category, items_per_page=items_per_page, sort_order=sort_order)
        page_contents = cls.fetch_page_content(page_url)
        listing_data = cls.parse_listing_data(page_contents)

        if max_pages > 1:
            total_item_counts = int(page_contents.find('h1', class_='srp-controls__count-heading')\
                                    .text.strip().replace(',','').replace('+','').split(' ',1)[0])
            total_pages = min(max_pages, int(total_item_counts/items_per_page))

            for _page in range(2,total_pages+1):
                _url = cls.build_search_url(query, page=_page, category=category, items_per_page=items_per_page, sort_order=sort_order)
                _contents = cls.fetch_page_content(_url)
                cls.parse_listing_data(_contents, listing_data)

                if _page < total_pages:
                  time.sleep(2)

        date = datetime.now().strftime('%Y-%m-%d')
        cls.save_to_csv(listing_data,filename=f"{query.replace(' ','_')}_{date}.csv")


In [ ]:
query = 'iPhone 16'
EbayScraping.scrape_search_results(query, max_pages=3)